In [ ]:
# !pip install scikit-learn
# !pip install gensim
# !pip install nltk
# !pip install BeautifulSoup4
# !pip install pandas

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from gensim.models import word2vec
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import logging
import nltk
import time
import re
# nltk.download()

In [2]:
USE_ADDED_DATA = 1

## Load data

In [3]:
train_labelled = pd.read_csv("word2vec-nlp-tutorial/labeledTrainData.tsv", delimiter="\t")
df2 = pd.read_csv('imdb_master.csv',encoding="latin-1")

df2 = df2.drop(['Unnamed: 0','type','file'],axis=1)
df2.columns = ["review","sentiment"]
df2 = df2[df2.sentiment != 'unsup']
df2['sentiment'] = df2['sentiment'].map({'pos': 1, 'neg': 0})

train = train_labelled.copy()
train = train.drop(['id'], axis=1)

train_added_data = pd.concat([train, df2]).reset_index(drop=True)

/Users/hardikmalhotra/anaconda2/envs/scikit/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [4]:
train_data = train_added_data if USE_ADDED_DATA else train

In [5]:
stops = set(stopwords.words("english"))

def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                                               
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

def get_clean_reviews(dataset):
    # Get the number of reviews based on the dataframe column size
    num_reviews = dataset["review"].size

    print ("Cleaning and parsing the training set movie reviews...\n")
    clean_train_reviews = []
    for i in range( 0, num_reviews ):
        # If the index is evenly divisible by 1000, print a message
        if( (i+1)%1000 == 0 ):
            print ("Review %d of %d\n" % ( i+1, num_reviews ) )                                                                  
        clean_train_reviews.append( review_to_words( dataset["review"][i] ))
        
    return clean_train_reviews

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 6000) 

In [6]:
clean_reviews = get_clean_reviews(train_data)
print("Vectorizing clean reviews")
train_data_features = vectorizer.fit_transform(clean_reviews)
train_data_features = train_data_features.toarray()

train_data_features.shape

Cleaning and parsing the training set movie reviews...

Review 1000 of 75000

Review 2000 of 75000

Review 3000 of 75000

Review 4000 of 75000

Review 5000 of 75000

Review 6000 of 75000

Review 7000 of 75000

Review 8000 of 75000

Review 9000 of 75000

Review 10000 of 75000

Review 11000 of 75000

Review 12000 of 75000

Review 13000 of 75000

Review 14000 of 75000

Review 15000 of 75000

Review 16000 of 75000

Review 17000 of 75000

Review 18000 of 75000

Review 19000 of 75000

Review 20000 of 75000

Review 21000 of 75000

Review 22000 of 75000

Review 23000 of 75000

Review 24000 of 75000

Review 25000 of 75000

Review 26000 of 75000

Review 27000 of 75000

Review 28000 of 75000

Review 29000 of 75000

Review 30000 of 75000

Review 31000 of 75000

Review 32000 of 75000

Review 33000 of 75000

Review 34000 of 75000

Review 35000 of 75000

Review 36000 of 75000

Review 37000 of 75000

Review 38000 of 75000

Review 39000 of 75000

Review 40000 of 75000

Review 41000 of 75000

Review 420

(75000, 6000)

## Evaluation on train and val data

In [7]:
def evaluate_train_val(train_data_features, train_data):
    x_train,x_test,y_train,y_test=train_test_split(train_data_features,train_data["sentiment"],test_size=0.2,random_state=0)
    print("Splitting Done")

    forest = RandomForestClassifier(n_estimators = 100) 

    forest = forest.fit(x_train, y_train )

    result = forest.predict(x_test)
    print("Predictions Done")

    accuracy=accuracy_score(y_test,result)

    cm =confusion_matrix(y_test,result)

    f1 =f1_score(y_test,result)

    print("f1 : {f1}\ncm :\n {cm}\naccuracy : {acc}".format(f1 = f1, cm = cm, acc = accuracy))

In [8]:
evaluate_train_val(train_data_features, train_data)

Splitting Done
Predictions Done
f1 : 0.9327641408751334
cm :
 [[7000  483]
 [ 525 6992]]
accuracy : 0.9328


## Supervised learning with Bag of Words

In [9]:
forest = RandomForestClassifier(n_estimators = 100) 

forest = forest.fit( train_data_features, train_data["sentiment"] )

## predict on test

In [10]:
# Read the test data
test = pd.read_csv("word2vec-nlp-tutorial/testData.tsv", header=0, delimiter="\t")

test_clean_reviews = get_clean_reviews(test)
test_data_features = vectorizer.transform(test_clean_reviews)

result = forest.predict(test_data_features)

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "submissions/Bag_of_Words_model_added_data.csv", index=False, quoting=3 )

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



## train word2vec

In [11]:
# Read data from files 
unlabeled_train = pd.read_csv( "word2vec-nlp-tutorial/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [12]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    
for review in df2["review"]:
    sentences += review_to_sentences(review, tokenizer)
    
    
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "models/300features_40minwords_10context_added_data"
model.save(model_name)

Parsing sentences from training set


/Users/hardikmalhotra/.local/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/hardikmalhotra/.local/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.happierabroad.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/hardikmalhotra/.local/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/hardikmalhotra/.local/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/hardikmalhotra/.local/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use 

In [ ]:
model = Word2vec.load("models/300features_40minwords_10context_added_data")

## Review -> features 

In [14]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        if counter % 1000 == 0:
            print("Review %d of %d " % (counter, len(reviews)))

        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        
        counter += 1 
    return reviewFeatureVecs

clean_train_reviews = []
for review in train_data["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 75000 


/Users/hardikmalhotra/anaconda2/envs/scikit/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 75000 
Review 2000 of 75000 
Review 3000 of 75000 
Review 4000 of 75000 
Review 5000 of 75000 
Review 6000 of 75000 
Review 7000 of 75000 
Review 8000 of 75000 
Review 9000 of 75000 
Review 10000 of 75000 
Review 11000 of 75000 
Review 12000 of 75000 
Review 13000 of 75000 
Review 14000 of 75000 
Review 15000 of 75000 
Review 16000 of 75000 
Review 17000 of 75000 
Review 18000 of 75000 
Review 19000 of 75000 
Review 20000 of 75000 
Review 21000 of 75000 
Review 22000 of 75000 
Review 23000 of 75000 
Review 24000 of 75000 
Review 25000 of 75000 
Review 26000 of 75000 
Review 27000 of 75000 
Review 28000 of 75000 
Review 29000 of 75000 
Review 30000 of 75000 
Review 31000 of 75000 
Review 32000 of 75000 
Review 33000 of 75000 
Review 34000 of 75000 
Review 35000 of 75000 
Review 36000 of 75000 
Review 37000 of 75000 
Review 38000 of 75000 
Review 39000 of 75000 
Review 40000 of 75000 
Review 41000 of 75000 
Review 42000 of 75000 
Review 43000 of 75000 
Review 44000 of 7500

## Evaluation in train data

In [15]:
evaluate_train_val(trainDataVecs, train_data)

Splitting Done
Predictions Done
f1 : 0.9250645994832042
cm :
 [[6888  595]
 [ 536 6981]]
accuracy : 0.9246


## Fit model

In [16]:
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainDataVecs, train_data["sentiment"] )

# Test & extract results 
result = forest.predict(testDataVecs)

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "submissions/Word2Vec_AverageVectors_added_data.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


## Review -> Clustering features

In [18]:
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

/Users/hardikmalhotra/anaconda2/envs/scikit/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


Time taken for K Means clustering:  2435.9109950065613 seconds.


In [19]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [20]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [21]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train_data["review"].size, num_clusters), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

## Evaluation

In [22]:
evaluate_train_val(train_centroids, train_data)

Splitting Done
Predictions Done
f1 : 0.9325662295959325
cm :
 [[7022  461]
 [ 547 6970]]
accuracy : 0.9328


In [ ]:
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train_data["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "submissions/BagOfCentroids.csv", index=False, quoting=3 )